In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

### Data

"Language.txt" 라는 18개의 텍스트 파일이 포함되어 있습니다. 각 파일에는 한 줄에 하나의 이름이 포함되어 있으며 대부분 로마자로 표시되어 있습니다.
각 언어 별 이름 목록 사전 `{language : [name ...]}`을 만듭니다. 일반 변수 category와 line은 이후의 확장성을 위해 사용됩니다. 

In [2]:
def findFiles(path): return glob.glob(path)
print(findFiles('data/names/*.txt'))

['data/names/Czech.txt', 'data/names/German.txt', 'data/names/Arabic.txt', 'data/names/Japanese.txt', 'data/names/Chinese.txt', 'data/names/Vietnamese.txt', 'data/names/Russian.txt', 'data/names/French.txt', 'data/names/Irish.txt', 'data/names/English.txt', 'data/names/Spanish.txt', 'data/names/Greek.txt', 'data/names/Italian.txt', 'data/names/Portuguese.txt', 'data/names/Scottish.txt', 'data/names/Dutch.txt', 'data/names/Korean.txt', 'data/names/Polish.txt']


In [3]:
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

In [4]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD',s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

Slusarski


In [5]:
# 각 언어별 이름 목록인 category_line 사전 생성
category_lines = {}
all_categories = []

# 파일을 읽고 줄 단위로 분리
def readLine(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLine(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

### Name to Tensor

하나의 문자를 표현하기 위해, one-hot vector를 사용합니다.<1 x n_letter> 문자의 index에 1 이고 나머지는 0으로 채워집니다. 단어를 만들기 위해서 그것을 2차원에 결합시킵니다. <line_length x 1 x n_letter>에 결합시킵니다. 추가 1차원은 Pytorch 가 모든것이 batch가 있다고 가정하게 됩니다.

In [6]:
import torch

In [7]:
n_letter = "seungehondoh"
tensor = torch.zeros(1, n_letters)

In [8]:
tensor

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])

In [9]:
import torch
import torch.nn as nn

# all_letter로 문자의 주소를 찾습니다.
def letterToIndex(letter):
    return all_letters.find(letter)

# 하나의 문자를 <1 x n_letters> Tensor로 변환시킨다.
def letterToTensor(letter):
    # 공간만큼의 0을 만들어 두고
    tensor = torch.zeros(1, n_letters)
    # 해당위치의 index를 1로 바꿔어주자
    tensor[0][letterToIndex(letter)] =1
    return tensor

# 한 줄(이름)을 <line_length x 1 x n_letters>.
# 문자 백터의 array로 변경한다.
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


In [10]:
letterToTensor('J')

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])

In [11]:
lineToTensor('Jones').size()

torch.Size([5, 1, 57])

In [12]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [24]:
input_one = letterToTensor("A")
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input_one,hidden)
output

tensor([[-2.9765, -2.8904, -2.8929, -3.0187, -2.9175, -2.8528, -2.9089, -2.8859,
         -2.8955, -2.8218, -2.9705, -2.8980, -2.8514, -2.8626, -2.8734, -2.7787,
         -2.8793, -2.8782]], grad_fn=<LogSoftmaxBackward>)

In [25]:
input_sen = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input_sen[0], hidden)
output

tensor([[-2.9765, -2.8904, -2.8929, -3.0187, -2.9175, -2.8528, -2.9089, -2.8859,
         -2.8955, -2.8218, -2.9705, -2.8980, -2.8514, -2.8626, -2.8734, -2.7787,
         -2.8793, -2.8782]], grad_fn=<LogSoftmaxBackward>)

In [31]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Dutch', 15)


In [46]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l)-1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype= torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

In [51]:
for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print("Category: ",category, 
          "line: ", line)

Category:  Dutch line:  Penner
Category:  English line:  Elkin
Category:  Vietnamese line:  Luc
Category:  Chinese line:  Cao
Category:  Czech line:  Kosko
Category:  Dutch line:  Meeuwissen
Category:  Czech line:  Koza
Category:  Spanish line:  Reyes
Category:  Vietnamese line:  Ma
Category:  Spanish line:  Jorda


In [52]:
## nn.LogSoftmax 가 손실함수이다.
criterion = nn.NLLLoss()

- input과 target tensor 생성
- 0으로 초기화된 은닉 상태 생성
- 문자 읽기
    - 다음 문자를 위한 은닉 상태유지
- 목표와 출력 비교
- 역전파
- 출력과 손실 리턴

In [53]:
learning_rate = 0.005

In [54]:
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    
    rnn.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        
    loss= criterion(output, category_tensor)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
        
    return output, loss.item()

In [62]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# 도식화를 위한 소실 추적
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # iter 숫자, 손실, 이름, 추측 출력
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # 현재 평균 손실을 손실 리스트에 추가
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 10s) 2.0333 Traverse / Spanish ✗ (French)
10000 10% (0m 18s) 1.2978 Sanchez / Spanish ✓
15000 15% (0m 26s) 0.3647 Hajjar / Arabic ✓
20000 20% (0m 34s) 1.0870 Yi / Korean ✓
25000 25% (0m 42s) 0.4029 Kanesaka / Japanese ✓
30000 30% (0m 52s) 0.4736 O'Donnell / Irish ✓
35000 35% (1m 2s) 1.6689 Daley / English ✓
40000 40% (1m 10s) 0.6353 Mikhail / Arabic ✓
45000 45% (1m 18s) 0.1732 Rudawski / Polish ✓
50000 50% (1m 26s) 1.7918 Alphen / Vietnamese ✗ (Dutch)
55000 55% (1m 34s) 1.4349 Aleppo / Portuguese ✗ (Italian)
60000 60% (1m 42s) 1.7155 She / Korean ✗ (Chinese)
65000 65% (1m 49s) 2.1323 Kwei / Polish ✗ (Chinese)
70000 70% (1m 57s) 2.7243 Haanrath / English ✗ (Dutch)
75000 75% (2m 5s) 1.7902 Sokal / Czech ✗ (Polish)
80000 80% (2m 12s) 0.0643 Kurofuji / Japanese ✓
85000 85% (2m 21s) 0.3198 Ramakers / Dutch ✓
90000 90% (2m 29s) 1.0557 Michael / Irish ✓
95000 95% (2m 37s) 0.9265 Stedronsky / Polish ✗ (Czech)
100000 100% (2m 45s) 0.1061 Mitchell / Scottish ✓
